In [125]:
import numpy as np
import re
from sklearn import linear_model

from coreNlpUtil import *
subject_word_to_indx = { "ship":2,"boy":4 }
action_word_to_indx = {"sail" : 3 }
object_word_to_indx = {"sea" : 5 }
vocab_count = 6
partial_captions = [(2,3,5)]
cap = np.array([0,1,2,3,4]).reshape((5,1))
images_features = np.arange(5000).reshape((5,1000))

In [22]:

n_samples = len(images_features)

X_train = images_features
y_train = cap

logistic = linear_model.LogisticRegression()

print('Logistic Regression score: %f'
      % logistic.fit(X_train, y_train).score(X_train, y_train))
logistic.predict(X_train)

Logistic Regression score: 1.000000


array([0, 1, 2, 3, 4])

In [17]:
import pickle
d = pickle.load(open('flickr8k_Entire_dataset_with_cnn_features.pkl'))

In [18]:
for i in (d[:10]):
    print i['sentences'][0]['raw']


A black dog is running after a white dog in the snow .
A little baby plays croquet .
A brown dog in the snow has something hot pink in its mouth .
A brown dog is running along a beach .
A black and white dog with a red Frisbee standing on a sandy beach .
A cyclist wearing a red helmet is riding on the pavement .
A man dressed in a purple shirt and red bandanna smiles at the people watching him .
A boy wearing a red t-shirt is running through woodland .
A girl in a white dress .
A skier in a yellow jacket is airborne above the mountains .


In [ ]:
"dog"
"run"
"snow"

"baby" 
"play"
"truck"

"dog"
"hold"
"snow"


"dog" 
"run"
"beach"


"dog"
"stand"
"beach"

"cyclist"
"ride"
"pavement"


A man dressed in a purple shirt and red bandanna smiles at the people watching him .
A boy wearing a red t-shirt is running through woodland .
A girl in a white dress .
A skier in a yellow jacket is airborne above the mountains .

In [36]:
subj_word_dict = {"dog":1,"baby":2,"cyclist":3}
action_word_dict = {"run":1,"play":2, "hold":3, "stand":4, "ride":5}
area_word_dict = {"snow":1, "truck":2, "beach": 3, "pavement":4}
y_sub = np.array([1,2,1,1,1,3])
y_action = np.array([1,2,3,1,4,5])
y_area = np.array([1,2,1,3,3,4])

In [34]:
X_train = []
for i in (d[:6]):
    X_train.append(i['cnn features'])
X_train = np.array(X_train)

In [37]:
logistic_subject = linear_model.LogisticRegression()
logistic_action = linear_model.LogisticRegression()
logistic_area = linear_model.LogisticRegression()

print('Logistic Regression score: %f'
      % logistic_subject.fit(X_train, y_sub).score(X_train, y_sub))

print('Logistic Regression score: %f'
      % logistic_action.fit(X_train, y_action).score(X_train, y_action))

print('Logistic Regression score: %f'
      % logistic_area.fit(X_train, y_area).score(X_train, y_area))

Logistic Regression score: 0.666667
Logistic Regression score: 0.666667
Logistic Regression score: 0.666667


In [40]:
logistic_action.predict(X_train)

array([1, 1, 3, 1, 4, 1])

In [98]:
import nltk

def remove_char_dep_res(text):
    return re.sub("{.*}","",text)

def process_text(triplet):
    return re.sub("{.*}","",triplet[0]), re.sub("{.*}","",triplet[1]), re.sub("{.*}","",triplet[2])
    #triplet[1] = 
    #triplet[2] = re.sub("{.*}","",triplet[2])
    #return triplet

def find_category_pos(pos):
    if(pos in ["VBG"]):
        return "Verb"
    if(pos in ["NN"]):
        return "Noun"
    return "UNKNOWN Category"

def find_triplet_nltk(root, prep_parses, dep_parse):
    Verb = []
    Noun = []
    if len(root) == 0:
        return None
    for prep_parse in prep_parses:
        if(root == prep_parse[1]):
            word = (remove_char_dep_res(prep_parse[0]))
            #print word
            #print find_category_pos(nltk.pos_tag([word])[0][1])
            if(find_category_pos(nltk.pos_tag([word])[0][1]) == "Verb"):
                
                Verb.append(word)
            if(find_category_pos(nltk.pos_tag([word])[0][1]) == "Noun"):
                Noun.append(word)
            if(len(Verb) >= 1 and len(Noun) >= 1):
                break;
    if(len(Verb) >= 1 and len(Noun) >= 1):
        return (remove_char_dep_res(root), Verb[0], Noun[0])
    for parse in dep_parse:
        m = re.search("prep_(.*)", parse[0])
        if(m):
            return process_text((root, m.group(1), parse[2]))
        
    return None   

def triplet(dep_parse):
    nsubj = []
    dobj = []
    prep = []
    root = ""
    for parse in dep_parse:
        if  parse[0] == "nsubj":
            nsubj.append( (parse[2], parse[1]) )

        elif  parse[0] == "dobj":
            dobj.append( (parse[2], parse[1]) )

        elif re.search("prep_", parse[0]):
            prep.append((parse[2], parse[1]))

        elif parse[0] == "root":
            root = parse[2]
    found = False
    if(len(nsubj)>0):
        for nsubj_dep in nsubj:
            for dobj_dep in dobj:
                if(nsubj_dep[1] == dobj_dep[1]):
                    triplet = process_text((nsubj_dep[0], nsubj_dep[1], dobj_dep[0]))
                    print triplet
                    found = True
                    break;
            if(found):
                break;
            
            for prep_dep in prep:
                if(nsubj_dep[1] == prep_dep[1]):
                    triplet = process_text((nsubj_dep[0], nsubj_dep[1], prep_dep[0]))
                    print triplet
                    found = True
                    break;
            if(found):
                break;
    else:
        if(len(root)>0):
            trip = find_triplet_nltk(root, prep, dep_parse)
            if(trip!= None):
                triplet = trip
                print triplet
                found = True
    if(found == False):
        print "SUBJECTS are "
        print nsubj
        print "OBJECTS are "
        print dobj
        print "PREP are "
        print prep
        print "Entire Dep Parse"
        print dep_parse
        return None
    else:
        return triplet


https://github.com/ma-sultan/monolingual-word-aligner
https://github.com/dasmith/stanford-corenlp-python
Follow above instructions in the monolingual word aligner including the one which states to modify the certain line number 100, THen RUN the Corenlp server and check

In [119]:
def process_dataset(dataset):
    Triplets = []
    X_train = []
    for i in (dataset):
        print i['sentences'][0]['raw']
        sentence1ParseResult = parseText(i['sentences'][0]['raw'])
        result = dependencyParseAndPutOffsets(sentence1ParseResult)
        triplet_value = triplet(result)
        Triplets.append(triplet_value)
        X_train.append(i['cnn features'])
        #print result
        print ""
    return np.array(X_train), Triplets

In [107]:
def process_triplet_dictionary(allTriplets):
    subj_vocab = []
    action_vocab = []
    object_vocab = []
    for triplet in allTriplets:
        subj_vocab.append(triplet[0])
        action_vocab.append(triplet[1])
        object_vocab.append(triplet[2])
    subj_vocab.append("UNK")
    action_vocab.append("UNK")
    object_vocab.append("UNK")
    subj_word_to_index = {w: i for i, w in enumerate(subj_vocab)}
    subj_index_to_word = {i: w for i, w in enumerate(subj_vocab)}
    action_word_to_index = {w: i for i, w in enumerate(action_vocab)}
    action_index_to_word = {i: w for i, w in enumerate(action_vocab)}
    object_word_to_index = {w: i for i, w in enumerate(object_vocab)}
    object_index_to_word = {i: w for i, w in enumerate(object_vocab)}
    #print subj_word_to_index,subj_index_to_word,action_word_to_index,action_index_to_word,object_word_to_index,object_index_to_word
    return subj_word_to_index,subj_index_to_word,action_word_to_index,action_index_to_word,object_word_to_index,object_index_to_word

In [123]:
def get_y_values(allTriplets,subj_word_to_index,action_word_to_index,object_word_to_index):
    y_subj = []
    y_action = []
    y_obj = []
    for triplet in allTriplets:
        if(triplet[0] in subj_word_to_index):
            y_subj.append(subj_word_to_index[triplet[0]])
        else:
            y_subj.append(subj_word_to_index["UNK"])
        if(triplet[1] in action_word_to_index):
            y_action.append(action_word_to_index[triplet[1]])
        else:
            y_action.append(action_word_to_index["UNK"])
        if(triplet[2] in object_word_to_index):
            y_obj.append(object_word_to_index[triplet[2]])
        else:
            y_obj.append(object_word_to_index["UNK"])
    return np.array(y_subj), np.array(y_action), np.array(y_obj)

def print_output(y_sub, y_action, y_obj, subj_index_to_word, action_index_to_word, object_index_to_word):
    for i in range(len(y_sub)):
        print subj_index_to_word[y_sub[i]], action_index_to_word[y_action[i]], object_index_to_word[y_obj[i]]

def train_model(X_train, y_sub, y_action, y_area):
    logistic_subject = linear_model.LogisticRegression()
    logistic_action = linear_model.LogisticRegression()
    logistic_area = linear_model.LogisticRegression()

    print('Logistic Regression score: %f'
          % logistic_subject.fit(X_train, y_sub).score(X_train, y_sub))

    print('Logistic Regression score: %f'
          % logistic_action.fit(X_train, y_action).score(X_train, y_action))

    print('Logistic Regression score: %f'
          % logistic_area.fit(X_train, y_area).score(X_train, y_area))
    return logistic_subject, logistic_action, logistic_area

In [120]:
X_train, Triplets = process_dataset(d[:20])
subj_word_to_index,subj_index_to_word,action_word_to_index,action_index_to_word,object_word_to_index,object_index_to_word = process_triplet_dictionary(Triplets)
y_sub_train,y_action_train,y_obj_train = get_y_values(Triplets,subj_word_to_index,action_word_to_index,object_word_to_index)
print_output(y_sub_train, y_action_train, y_obj_train, subj_index_to_word, action_index_to_word, object_index_to_word)

A black dog is running after a white dog in the snow .
(u'dog', u'running', u'dog')

A little baby plays croquet .
(u'baby', u'plays', u'croquet')

A brown dog in the snow has something hot pink in its mouth .
(u'dog', u'has', u'pink')

A brown dog is running along a beach .
(u'dog', u'running', u'beach')

A black and white dog with a red Frisbee standing on a sandy beach .
(u'dog', u'standing', u'beach')

A cyclist wearing a red helmet is riding on the pavement .
(u'cyclist', u'riding', u'pavement')

A man dressed in a purple shirt and red bandanna smiles at the people watching him .
(u'people', u'watching', u'him')

A boy wearing a red t-shirt is running through woodland .
(u'boy', u'running', u'woodland')

A girl in a white dress .
(u'girl', u'in', u'dress')

A skier in a yellow jacket is airborne above the mountains .
(u'skier', u'airborne', u'mountains')

A photographer looks over the hills .
(u'photographer', u'looks', u'hills')

A bunch of girls in cheerleader outfits .
(u'bunch

In [126]:
lr_s,lr_a,lr_o = train_model(X_train, y_sub_train, y_action_train, y_obj_train)

Logistic Regression score: 0.300000
Logistic Regression score: 0.300000
Logistic Regression score: 0.400000


In [128]:
y_pred_s = lr_s.predict(X_train)
y_pred_a = lr_a.predict(X_train)
y_pred_o = lr_o.predict(X_train)

In [129]:
print_output(y_pred_s, y_pred_a, y_pred_o, subj_index_to_word, action_index_to_word, object_index_to_word)

dog running dog
dog in dog
dog in pink
dog running beach
dog in beach
dog in dog
dog in dog
dog in dog
dog in dog
dog in mountains
dog looks hills
dog in girls
dog in dog
dog running dog
dog in dog
dog in trail
dog in dog
dog in dog
dog running beach
dog in dog
